In [3]:
# =========================================================
# 📘 DEPTH ESTIMATION TRAINING NOTEBOOK
# =========================================================

import os
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from data_loader import get_data_loaders
from models.model import DepthNet

# =========================================================
# Setup
# =========================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
if device.type == 'cuda':
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")

# Create unified outputs directory
base_dir = "outputs"
os.makedirs(os.path.join(base_dir, "checkpoints"), exist_ok=True)
os.makedirs(os.path.join(base_dir, "logs"), exist_ok=True)
os.makedirs(os.path.join(base_dir, "plots"), exist_ok=True)


Using device: cuda
✅ GPU: NVIDIA GeForce RTX 4060 Laptop GPU


In [4]:
# =========================================================
# Load Data
# =========================================================
train_loader, val_loader = get_data_loaders('data', pseudo_dir='pseudo_data', batch_size=4)


[INFO] Found 250 real KITTI pairs.
[INFO] Found 2706 pseudo-labeled pairs.
[INFO] Using 2956 image–depth pairs total.
[INFO] Using 2364 image–depth pairs for training.
[INFO] Using 592 image–depth pairs for validation.


In [5]:
# =========================================================
# Model, Loss, Optimizer
# =========================================================
model = DepthNet().to(device)
criterion = nn.L1Loss()  # MAE loss for depth
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
# =========================================================
# Training Loop
# =========================================================
num_epochs = 20
metrics = {"epoch": [], "train_loss": [], "val_loss": []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for imgs, depths in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        imgs, depths = imgs.to(device), depths.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, depths)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)

    # Validation Phase
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for imgs, depths in val_loader:
            imgs, depths = imgs.to(device), depths.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, depths)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)

    # Log metrics
    print(f"[Epoch {epoch+1}] Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
    metrics["epoch"].append(epoch + 1)
    metrics["train_loss"].append(avg_train_loss)
    metrics["val_loss"].append(avg_val_loss)

    # Save model checkpoint
    torch.save(
        model.state_dict(),
        os.path.join(base_dir, "checkpoints", f"depthnet_epoch{epoch+1}.pth")
    )

print(" Training Complete!")


Epoch 1/20: 100%|██████████| 591/591 [02:04<00:00,  4.74it/s]


[Epoch 1] Train Loss: 0.1627 | Val Loss: 0.1423


Epoch 2/20: 100%|██████████| 591/591 [01:59<00:00,  4.94it/s]


[Epoch 2] Train Loss: 0.1524 | Val Loss: 0.1400


Epoch 3/20: 100%|██████████| 591/591 [01:26<00:00,  6.80it/s]


...

[Epoch 18] Train Loss: 0.1347 | Val Loss: 0.1305


Epoch 19/20: 100%|██████████| 591/591 [01:23<00:00,  7.10it/s]


[Epoch 19] Train Loss: 0.1340 | Val Loss: 0.1316


Epoch 20/20: 100%|██████████| 591/591 [01:24<00:00,  7.03it/s]


[Epoch 20] Train Loss: 0.1337 | Val Loss: 0.1306
✅ Training Complete!


In [7]:
# =========================================================
# Save Metrics
# =========================================================
df = pd.DataFrame(metrics)
csv_path = os.path.join(base_dir, "logs", "training_metrics.csv")
df.to_csv(csv_path, index=False)
print(f"📄 Metrics saved to: {csv_path}")
df.head()


📄 Metrics saved to: outputs\logs\training_metrics.csv


,epoch,train_loss,val_loss
0,1,0.162707,0.142302
1,2,0.152404,0.139964
2,3,0.148956,0.141533
3,4,0.146729,0.137842
4,5,0.144648,0.132364


In [8]:
# =========================================================
# Plot & Save Loss Curves
# =========================================================
plt.figure(figsize=(8,5))
plt.plot(df["epoch"], df["train_loss"], label="Train Loss", marker='o')
plt.plot(df["epoch"], df["val_loss"], label="Val Loss", marker='s')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("DepthNet Training & Validation Loss")
plt.legend()
plt.grid(True)
plt.tight_layout()

plot_path = os.path.join(base_dir, "plots", "loss_curve.png")
plt.savefig(plot_path)
plt.close()
print(f"📊 Loss curve saved to: {plot_path}")


📊 Loss curve saved to: outputs\plots\loss_curve.png


In [10]:
# =========================================================
# 📸 Polished Visualization (Fixed Resolution + Formatting)
# =========================================================
import numpy as np
import matplotlib.pyplot as plt
import torch
import os

model.eval()
vis_dir = os.path.join(base_dir, "plots", "visualizations")
os.makedirs(vis_dir, exist_ok=True)

num_samples = 5  # number of samples to visualize
count = 0

with torch.no_grad():
    for imgs, depths in val_loader:
        imgs, depths = imgs.to(device), depths.to(device)
        outputs = model(imgs)

        for i in range(imgs.size(0)):
            if count >= num_samples:
                break

            rgb = imgs[i].permute(1, 2, 0).cpu().numpy()
            depth_gt = depths[i].squeeze().cpu().numpy()
            depth_pred = outputs[i].squeeze().cpu().numpy()

            fig, axes = plt.subplots(1, 3, figsize=(12, 4), dpi=100)
            axes[0].imshow(rgb)
            axes[0].set_title("RGB Image", fontsize=12)
            axes[0].axis("off")

            axes[1].imshow(depth_gt, cmap="plasma")
            axes[1].set_title("Ground Truth Depth", fontsize=12)
            axes[1].axis("off")

            axes[2].imshow(depth_pred, cmap="plasma")
            axes[2].set_title("Predicted Depth", fontsize=12)
            axes[2].axis("off")

            plt.subplots_adjust(wspace=0.02, hspace=0)
            save_path = os.path.join(vis_dir, f"depth_vis_{count+1}.png")
            plt.savefig(save_path, bbox_inches="tight", pad_inches=0)
            plt.close(fig)

            print(f"🖼️ Saved visualization: {save_path}")
            count += 1

        if count >= num_samples:
            break

print("✅ All visualizations saved to:", vis_dir)


🖼️ Saved visualization: outputs\plots\visualizations\depth_vis_1.png
🖼️ Saved visualization: outputs\plots\visualizations\depth_vis_2.png
🖼️ Saved visualization: outputs\plots\visualizations\depth_vis_3.png
🖼️ Saved visualization: outputs\plots\visualizations\depth_vis_4.png
🖼️ Saved visualization: outputs\plots\visualizations\depth_vis_5.png
✅ All visualizations saved to: outputs\plots\visualizations
